In [ ]:
from PIL import Image
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pickle
from google.colab import drive
drive.mount('./gdrive', force_remount=True)

In [ ]:
# 구글 드라이브 ./gdrive/My Drive/Images/des_pickles 에 
# DES ecb, cbc, ofb 128*128 픽셀 이미지 1,2,16 라운드 이미지 array 저장
# 16라운드 암호화 진행 시 CBC, OFB는 육안으로 구분 불가

In [ ]:
# pickle 불러오기
with open('./gdrive/My Drive/Images/des_pickles/des_ecb_16r.pickle', 'rb') as f:
  ecb16r = pickle.load(f)
  f.close()
with open('./gdrive/My Drive/Images/des_pickles/des_cbc_16r.pickle', 'rb') as f:
  cbc16r = pickle.load(f)
  f.close()

In [ ]:
# 정답 레이블 만들기
# 0 : ECB, 1 : CBC
# CBC, OFB는 구분 불가하므로 ECB와 CBC 만을 이용해 학습
ecb16r_label = [0 for _ in range(1000)]
cbc16r_label = [1 for _ in range(1000)]
label = ecb16r_label + cbc16r_label
label = np.array(label)

In [ ]:
# ECB, CBC 데이터 합치고 섞기
data = np.concatenate((ecb16r, cbc16r), axis=0)

In [ ]:
import random

tmp = [[x, y] for x, y in zip(data, label)]
random.shuffle(tmp)
X = [n[0] for n in tmp]
Y = [n[1] for n in tmp]
X = np.array(X)
Y = np.array(Y)

In [ ]:
# 2000개 데이터 200개 검증용, 1800개 훈련용으로, 255로 나눠서 이미지로
x_valid, x_train = X[:200] / 255, X[200:] / 255
y_valid, y_train = Y[:200], Y[200:]

In [ ]:
# 기존 AES128 모델로 학습시킨 것과 동일한 모델
model = keras.models.Sequential([
              keras.layers.Conv2D(32, 3, activation="relu", padding="same", input_shape=[128,128,3]),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(64, 3, activation="relu", padding="same"),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
              keras.layers.MaxPooling2D(2, padding="valid"),
              keras.layers.Dropout(0.25),
              keras.layers.Flatten(),
              keras.layers.Dense(512, activation="relu"),
              keras.layers.Dropout(0.5),
              keras.layers.Dense(2, activation="softmax")
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs=30, validation_data=(x_valid, y_valid))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(acc, label="accuracy")
plt.plot(val_acc, label="val_accuracy")
plt.plot(loss, label="loss")
plt.plot(val_loss, label="val_loss")
plt.xlabel('epoch')
plt.ylabel('ratio')
plt.ylim(0, 1.0)
plt.legend()
plt.show()

In [ ]:
# OFB, CBC 16round 판별 유무 확인
with open('./gdrive/My Drive/Images/des_pickles/des_ofb_16r.pickle', 'rb') as f:
  ofb16r = pickle.load(f)
  f.close()

In [ ]:
# 정답 레이블 만들기
# 0 : OFB, 1 : CBC
ofb16r_label = [0 for _ in range(1000)]
cbc16r_label = [1 for _ in range(1000)]
label2 = ofb16r_label + cbc16r_label
label2 = np.array(label2)

In [ ]:
# OFB, CBC 데이터 합치고 섞기
data2 = np.concatenate((ofb16r, cbc16r), axis=0)

In [ ]:
tmp2 = [[x, y] for x, y in zip(data2, label2)]
random.shuffle(tmp2)
X2 = [n[0] for n in tmp2]
Y2 = [n[1] for n in tmp2]
X2 = np.array(X2)
Y2 = np.array(Y2)

In [ ]:
# 2000개 데이터 200개 검증용, 1800개 훈련용으로, 255로 나눠서 이미지로
x_valid2, x_train2 = X2[:200] / 255, X2[200:] / 255
y_valid2, y_train2 = Y2[:200], Y2[200:]

In [ ]:
history2 = model.fit(x_train2, y_train2, epochs=30, validation_data=(x_valid2, y_valid2))

In [ ]:
acc2 = history2.history['accuracy']
val_acc2 = history2.history['val_accuracy']
loss2 = history2.history['loss']
val_loss2 = history2.history['val_loss']
plt.plot(acc2, label="accuracy")
plt.plot(val_acc2, label="val_accuracy")
plt.plot(loss2, label="loss")
plt.plot(val_loss2, label="val_loss")
plt.xlabel('epoch')
plt.ylabel('ratio')
plt.ylim(0, 1.0)
plt.legend()
plt.show()